<a href="https://colab.research.google.com/github/kazzand/TextEmbeddings/blob/master/Doc2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
!pip3 install pymorphy2

     |████████████████████████████████| 51kB 2.0MB/s 
     |████████████████████████████████| 7.1MB 8.1MB/s 


In [4]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
import pymorphy2
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
import numpy as np


nltk.download("stopwords")
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
data = pd.read_csv('/content/gdrive/My Drive/lenta-ru-news.csv', header = 0)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
data.shape

(800975, 6)

In [0]:
small_data = data[:10000]

In [8]:
small_data.head()

,url,title,text,topic,tags,date
0,https://lenta.ru/news/1914/09/16/hungarnn/,1914. Русские войска вступили в пределы Венгрии,Бои у Сопоцкина и Друскеник закончились отступ...,Библиотека,Первая мировая,1914/09/16
1,https://lenta.ru/news/1914/09/16/lermontov/,1914. Празднование столетия М.Ю. Лермонтова от...,"Министерство народного просвещения, в виду про...",Библиотека,Первая мировая,1914/09/16
2,https://lenta.ru/news/1914/09/17/nesteroff/,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в...",Библиотека,Первая мировая,1914/09/17
3,https://lenta.ru/news/1914/09/17/bulldogn/,1914. Бульдог-гонец под Льежем,Фотограф-корреспондент Daily Mirror рассказыва...,Библиотека,Первая мировая,1914/09/17
4,https://lenta.ru/news/1914/09/18/zver/,1914. Под Люблином пойман швабский зверь,"Лица, приехавшие в Варшаву из Люблина, передаю...",Библиотека,Первая мировая,1914/09/18


preprocess train data

In [0]:
def preprocess_text(data):
  corpus= []
  for text in data['text']:
    reg = re.compile('[^a-zA-Zа-яА-Я. ]')
    corpus.append(reg.sub('', text))
  for i in range(len(corpus)):
    corpus[i] = corpus[i].lower()
  print('lower is fine')
  sentences = []
  for news in corpus:
    sentences.append(nltk.sent_tokenize(news)) 

  print('sentences tokenize is fine')
  russian_stopwords = stopwords.words("russian")

  words_bag = []
  for mes in sentences:
    appender = []
    for sentence in mes:
      reg = re.compile('[^a-zA-Zа-яА-Я ]')
      a = (reg.sub('', sentence))
      mas = lemma(a)
      for el in mas:
        appender.append(el)
    appender_sw = [word for word in appender if not word in russian_stopwords]
    words_bag.append(appender_sw)

  print('Everything is fine')
  return words_bag

In [0]:
git clone https://kazzand@bitbucket.org/ArtyomBelov/news.git

SyntaxError: ignored

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
def lemma(text):
    morph = pymorphy2.MorphAnalyzer()  
    words = text.split() # разбиваем текст на слова
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)

    return res


In [10]:
words_bag = preprocess_text(small_data)

lower is fine
sentences tokenize is fine
Everything is fine


In [0]:
num = np.array(words_bag)
np.save('/content/gdrive/My Drive/lenta_tokenized.npy', num)

## **Doc2Vec**

In [0]:
words_bag = np.load('/content/gdrive/My Drive/lenta_tokenized.npy', allow_pickle=True)

In [0]:
train_data = []
for i, d in enumerate(words_bag):
    train_data.append(TaggedDocument(words = words_bag[i], tags=[str(i)]))



In [11]:
len(train_data)

10000

In [0]:
model = Doc2Vec(vector_size=50, min_count=1, epochs=40)
model.build_vocab(train_data)
model.train(train_data, total_examples=model.corpus_count, epochs=model.epochs)

In [0]:
v1 = model.infer_vector('путин объявил следующую неделю нерабочей из-за коронавируса отдыхайте лечитесь сидите дома'.split(' '))
v2 = model.infer_vector('следующая неделя каникулы об это на заседании объявил путин мы должны изолироваться чтоб избежать заражения коронавирусом'.split(' '))
v3 = model.infer_vector('собака любит кости и играть с мячиком кататься на скейтборде и есть бананы'.split(' '))


In [14]:
def cos(v1,v2):
        return v1 @ v2 / (np.linalg.norm(v1) * np.linalg.norm(v2))
print(cos(v1,v2))
print(cos(v3,v2))
print(cos(v1,v3))

0.7444042
-0.060164582
0.16586053


In [0]:
a1 = 'Президент Российской Федерации Владимир Владимирович Путин объявил следующую неделю выходной. Из-за опасности массового заражения людей рекомендуется избегать мест скопления людей, носить маски и костюмы химзащиты'
a2 = 'Неделя с 28 марта по 5 апреля объявлена выходной. Президент России Путин пожелал людям не болеть и оставаться дома'
a3 = 'Илон Маск запустил уже 8 спутников SpaceX и не собирается останавливаться, бесплатный интернет из любой точки планеты'

In [0]:
def preprocess_input_text(string):
  reg = re.compile('[^a-zA-Zа-яА-Я. ]')
  corpus = reg.sub('', string)
  corpus = corpus.lower()
  sentences = nltk.sent_tokenize(corpus) 

  morph = pymorphy2.MorphAnalyzer()  
  russian_stopwords = stopwords.words("russian")

  words_bag = []
  appender = []
  for sentence in sentences:
    reg = re.compile('[^a-zA-Zа-яА-Я ]')
    a = (reg.sub('', sentence))
    mas = lemma(a)
    
    appender_sw = [word for word in mas if not word in russian_stopwords]
    for el in appender_sw:
      words_bag.append(el)

  return words_bag

In [17]:
print(a1, '\n', a2, '\n', a3)

Президент Российской Федерации Владимир Владимирович Путин объявил следующую неделю выходной. Из-за опасности массового заражения людей рекомендуется избегать мест скопления людей, носить маски и костюмы химзащиты 
 Неделя с 28 марта по 5 апреля объявлена выходной. Президент России Путин пожелал людям не болеть и оставаться дома 
 Илон Маск запустил уже 8 спутников SpaceX и не собирается останавливаться, бесплатный интернет из любой точки планеты


In [0]:
v1 = model.infer_vector(preprocess_input_text(a1))
v2 = model.infer_vector(preprocess_input_text(a2))
v3 = model.infer_vector(preprocess_input_text(a3))


In [23]:
print(cos(v1,v2))
print(cos(v3,v2))
print(cos(v1,v3))

0.56147414
0.4243226
0.32267466


## **TF-IDF**

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf  = TfidfVectorizer()
corpus = tfidf.fit_transform(small_data['text'])

In [0]:
sen1 = list('На дворе зима цветы растут')
sen2 = list('луна луна цветы цветы люблю')
vec1 = tfidf.transform(sen1)
vec2 = tfidf.transform(sen2)

In [23]:
vec.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [0]:
import numpy as np
a1 = np.array(corpus[0].todense())
a2 = np.array(corpus[1].todense())

In [0]:
def cos(v1,v2):
        return v1 @ v2 / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [25]:
print(cos(vec1,vec2))

ValueError: ignored

## ruBERT

In [0]:
import numpy
import requests
import tqdm

In [20]:
!pip3 install transformers
!pip3 install deeppavlov

     |████████████████████████████████| 542kB 3.4MB/s 
     |████████████████████████████████| 1.0MB 16.6MB/s 
     |████████████████████████████████| 870kB 20.8MB/s 
     |████████████████████████████████| 3.7MB 31.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=62e28679e2d3c07555442a8a9407601f0d6d01cf23a35aab3a079c5e834cbefa
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses
     |████████████████████████████████| 757kB 3.4MB/s 
     |████████████████████████████████| 51kB 5.9MB/s 
     |████████████████████████████████| 1.5MB 46.2MB/s 
     |████████████████████████████████| 8.0MB 30.5MB/s 
     |████████████████████████████████| 71kB 7.8MB/s 
     |████████████████████████████████| 61kB 7.0MB/s 
     |████████████████████████████████| 6.7MB 9.3MB/s 
     |████████████████████████████████| 2.1MB 26.5MB/s 
     |████████████████████████████████|

In [26]:
from deeppavlov.models.embedders.elmo_embedder import ELMoEmbedder
elmo = ELMoEmbedder("http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-news_wmt11-16_1.5M_steps.tar.gz")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Unzipping misc/perluniprops.zip.
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping corpora/nonbreaking_prefixes.zip.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [0]:
vec1 = elmo(['путин объявил следующую неделю выходной'.split(' ')])[0]
vec2 = elmo(['президент россии путин ввел режим чрезвычайной ситуации'.split(' ')])[0]
vec3 = elmo(['каравай каравай кого хочешь выбирай'.split(' ')])[0]


In [33]:
print(cos(vec1,vec2))
print(cos(vec3,vec2))
print(cos(vec1,vec3))

0.520512
0.18707918
0.3275849


## another deeppavlov

In [0]:
from transformers import AutoTokenizer, AutoModelWithLMHead

In [5]:
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

model = AutoModelWithLMHead.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

In [0]:
import torch
import numpy as np

In [11]:
sentence1 = 'Путин объявил следующую неделю выходной'
sentence2 = 'путин молодец, следующая неделя будет каникулярная'
sentence3 = 'чтоб не идти вдоль ночных дорог'

with torch.no_grad():
    input_ids = torch.tensor([tokenizer.encode(sentence1, add_special_tokens=True)]) 
    last_hidden_states = model(input_ids)[0]
    v1 = torch.mean(last_hidden_states[0],dim=0)
    
    input_ids = torch.tensor([tokenizer.encode(sentence2, add_special_tokens=True)]) 
    last_hidden_states = model(input_ids)[0]
    v2 = torch.mean(last_hidden_states[0],dim=0)

    input_ids = torch.tensor([tokenizer.encode(sentence3, add_special_tokens=True)]) 
    last_hidden_states = model(input_ids)[0]
    v3 = torch.mean(last_hidden_states[0],dim=0)

    v1 = np.array(v1.detach())
    v2 = np.array(v2.detach())
    v3 = np.array(v3.detach())
    print(len(v1))

119547


In [11]:
def cos(v1,v2):
        return v1 @ v2 / (np.linalg.norm(v1) * np.linalg.norm(v2))
    print(cos(v1,v2))
    print(cos(v3,v2))
    print(cos(v1,v3))

IndentationError: ignored